# Each State Graphed

#### Import Libraries

In [1]:
import pandas as pd
from datetime import datetime
import random
from bokeh.plotting import figure
from bokeh.layouts import row
from bokeh.models import LinearAxis, Range1d, HoverTool, CustomJS
from bokeh.models.widgets import CheckboxGroup
from bokeh.io import output_file, show
from bokeh.resources import CDN
from bokeh.embed import file_html

#### Read in .csv file and store in DataFrame

In [2]:
file = "us-counties.csv"
df = pd.read_csv(file)
df = df.drop(columns=['county','fips'])
df = df.groupby(['date','state'], as_index=False).sum()

#### Get colors from colors.txt

In [3]:
colorList = []
colorlist = open("colors.txt","r")
for color in colorlist:
    colorList.append(color)
colorlist.close()

#### Figure Graph

In [4]:
plotWidth = 1000
plotHeight = 500
startRange = 0
endRange = int(df['cases'].max()*.8)
graphs = figure(title = "States", x_axis_label='DATE', y_axis_label='CONFIRMED CASES OF COVID-19', x_axis_type='datetime',y_range=(startRange,endRange),tools='pan,box_zoom,zoom_in,zoom_out,reset,save',plot_width=plotWidth, plot_height=plotHeight)
graphs.extra_y_ranges = {"plot": Range1d(start=startRange, end=endRange)}
graphs.add_layout(LinearAxis(y_range_name="plot", axis_label='DECEASED FROM COVID-19'), 'right')
graphs.sizing_mode = 'stretch_both'
graphs.left[0].formatter.use_scientific = False
graphs.right[0].formatter.use_scientific = False
graphs.add_tools(HoverTool(tooltips=[("", "$name"),("","$x{%F}: $y{int}")],formatters = {"$x": "datetime"},mode='mouse',toggleable=False)) 

#### Graph states

In [5]:
plot_Cases = []
plot_Deaths = []

for state in sorted(df['state'].unique()):
    stateDF = df[df['state'] == state]
    dates = pd.to_datetime(stateDF['date'], format='%Y-%m-%d')
    cases = stateDF['cases']
    deaths = stateDF['deaths']
    
    color = random.choice(colorList)
    color = color.strip('\n')
    
    plot_Cases.append(graphs.line(dates,cases,line_width=2,line_color=color,name=state,legend_label="Confirmed"))
    plot_Deaths.append(graphs.line(dates,deaths,line_width=2,line_dash='dashed',line_color=color,name=state, y_range_name="plot",legend_label="Deceased"))

#Finish figuring graph
graphs.legend.location = 'top_left'
checkboxes = CheckboxGroup(labels=sorted(df['state'].unique()))

#Linking with JavaScript
callback = CustomJS(code="""
                            var i;
                            for (i = 0; i < %i; i++)
                            {
                                if (cb_obj.active.includes(i))
                                {
                                    lineC[i].visible = true;
                                    lineD[i].visible = true;
                                }
                                else
                                {
                                    lineC[i].visible = false;
                                    lineD[i].visible = false;
                                }
                            }
                            """ % len(df['state'].unique()),
                    args={'lineC': plot_Cases, 'lineD': plot_Deaths})


checkboxes.js_on_click(callback)

#Show Graph
output_file("statesGraph.html")
show(row(graphs,checkboxes))

#Store as html
# html = file_html(row(graphs,checkboxes), CDN, "US States")
# fa = open("stateGraphs.txt","w")
# fa.write(html)
# fa.close()